In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sn
import time
%matplotlib notebook
sn.set_style("ticks",sn.axes_style({'axes.grid': True}))

def distance(a,b):
    d = np.sqrt( (b[0]-a[0])**2 + (b[1]-a[1])**2 + (b[2]-a[2])**2 )
    return d

In [ ]:
random = np.loadtxt('100k_weighted_random.dat')
data = np.loadtxt('100k_weighted_north_cmass.dat')
dt=data.transpose()
rt=random.transpose()

In [ ]:
#plt.figure()
#ax=plt.axes(projection="3d")
#ax.scatter(dt[0],dt[1],dt[3],s=2)

#plt.figure()
#plt.scatter(dt[0],dt[1],s=2,marker='H',alpha=0.1)

#plt.figure()
#plt.scatter(rt[0],rt[1],s=2,marker='H',alpha=0.1)



#plt.figure()
#zhist = plt.hist(dt[2],bins=75)

#plt.figure()
#cdhist =  plt.hist(dt[3],bins=75)

In [ ]:
# z=v/c , v=h_0*d , z*c = h_0*d -> d=(z*c)/h_0
# ra = theta
# dec = phi

def spherical_to_cartesian(ar,alt_cmd=None):
    
    newpts=[]
    
    if alt_cmd is None:
        
        for i in range(len(ar[0])):
            
            ra = np.deg2rad(ar[0,i])
            dec = np.deg2rad(ar[1,i])
            cmd=ar[3,i]

            ############################

            x=cmd*np.cos(ra)*np.cos(dec)
            y=cmd*np.sin(ra)*np.cos(dec)
            z=cmd*np.sin(dec)

            ############################


            xyz=[x,y,z]

            newpts.append(xyz)
        
    else:
        
        for i in range(len(ar[0])):
            
            ra = np.deg2rad(ar[0,i])
            dec = np.deg2rad(ar[1,i])
            cmd=alt_cmd[i]

            ############################

            x=cmd*np.cos(ra)*np.cos(dec)
            y=cmd*np.sin(ra)*np.cos(dec)
            z=cmd*np.sin(dec)

            ############################


            xyz=[x,y,z]

            newpts.append(xyz)

    

    
        
    return np.array(newpts).transpose()

In [ ]:
import astropy
from astropy.cosmology import WMAP9 as cosmo
from astropy.cosmology import FlatLambdaCDM

cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
cd = cosmo.comoving_distance(dt[2])
cdist = cd.value * 0.7

In [ ]:
d_conv=spherical_to_cartesian(dt,alt_cmd=cdist)

r_conv=spherical_to_cartesian(rt)

plt.figure()
ax=plt.axes(projection="3d")
ax.scatter(d_conv[0],d_conv[1],d_conv[2],s=2,alpha=0.05)

In [ ]:
ndivs = 30

rxpts = np.linspace(np.round(min(r_conv[0])),np.round(max(r_conv[0])),ndivs)
rypts = np.linspace(np.round(min(r_conv[1])),np.round(max(r_conv[1])),ndivs)
rzpts = np.linspace(np.round(min(r_conv[2])),np.round(max(r_conv[2])),ndivs)

dxpts = np.linspace(np.round(min(d_conv[0])),np.round(max(d_conv[0])),ndivs)
dypts = np.linspace(np.round(min(d_conv[1])),np.round(max(d_conv[1])),ndivs)
dzpts = np.linspace(np.round(min(d_conv[2])),np.round(max(d_conv[2])),ndivs)

In [ ]:
ddivs_master = {}

xwidth=dxpts[1]-dxpts[0]
ywidth=dypts[1]-dypts[0]
zwidth=dzpts[1]-dzpts[0]

x,y,z=d_conv[0],d_conv[1],d_conv[2]

ddivs = []

for a,i in enumerate(dxpts[range(ndivs)]):

    index_x0 = x>i
    index_x1 = x<= i+xwidth

    for b,j in enumerate(dypts[range(ndivs)]):

        index_y0 = y>j
        index_y1 = y<= j+ywidth
        
        for c,k in enumerate(dzpts[range(ndivs)]):
                  
            index_z0 = z>k
            index_z1 = z<= k+zwidth
            
            index = index_x0*index_x1 * index_y0*index_y1 * index_z0*index_z1
            
            xsub = x[index]
            ysub = y[index]
            zsub = z[index]
            
            ddivs.append([xsub,ysub,zsub])

            key = "%02d%02d%02d" % (a,b,c)
            ddivs_master[key] = [xsub,ysub,zsub]

In [ ]:
rdivs_master = {}

xwidth=rxpts[1]-rxpts[0]
ywidth=rypts[1]-rypts[0]
zwidth=rzpts[1]-rzpts[0]

x,y,z=r_conv[0],r_conv[1],r_conv[2]

rdivs = []

for a,i in enumerate(rxpts[range(ndivs)]):

    index_x0 = x>i
    index_x1 = x<= i+xwidth

    for b,j in enumerate(rypts[range(ndivs)]):

        index_y0 = y>j
        index_y1 = y<= j+ywidth
        
        for c,k in enumerate(rzpts[range(ndivs)]):
                  
            index_z0 = z>k
            index_z1 = z<= k+zwidth
            
            index = index_x0*index_x1 * index_y0*index_y1 * index_z0*index_z1
            
            xsub = x[index]
            ysub = y[index]
            zsub = z[index]
            
            rdivs.append([xsub,ysub,zsub])

            key = "%02d%02d%02d" % (a,b,c)
            rdivs_master[key] = [xsub,ysub,zsub]


In [ ]:
'''
plt.figure()
ax = plt.axes(projection='3d')

for i in range(ndivs):
    for j in range(ndivs):
        for k in range(ndivs):
            key = "%02d%02d%02d" % (i,j,k)
            ax.scatter(ddivs_master[key][0],ddivs_master[key][1],ddivs_master[key][2],s=20,marker='.')
'''
#plt.title('Party Sphere')
#ax.set_xlabel('x')
#ax.set_ylabel('y')
#ax.set_zlabel('z')

In [ ]:
######################################################################################
# nearest neighbor
######################################################################################

nsubs = ndivs-1

hist_dd = np.zeros(200)
hist_dr = np.zeros(200)
hist_rr = np.zeros(200)

start = time.time()
for i in range(nsubs):
    
    for j in range(nsubs):
        
        for k in range(nsubs):
            
            home = "%02d%02d%02d" % (i,j,k)

            for nni in range(i,i+2):
                
                for nnj in range(j,j+2):
                    
                    for nnk in range(k,k+2):
                        
                        if nni<nsubs and nnj<nsubs and nnk<nsubs:
                            nn = "%02d%02d%02d" % (nni,nnj,nnk)
                            #print("\t",nn)
                            
                            d_home_subdiv = np.array(ddivs_master[home]).transpose()
                            d_nn_subdiv = np.array(ddivs_master[nn]).transpose()
                            r_home_subdiv = np.array(rdivs_master[home]).transpose()
                            r_nn_subdiv = np.array(rdivs_master[nn]).transpose()
                            
                            for dpt in range(0,len(d_home_subdiv)):
                                pos1=d_home_subdiv[dpt]
                                pos2=d_nn_subdiv[:].transpose()
                                d_dist=distance(pos1,pos2)

                                dr_dist=distance(pos1,r_nn_subdiv[:].transpose())

                                h_dd = np.histogram(d_dist,bins=200,range=(0,200))
                                hist_dd += h_dd[0]
                                
                                h_dr = np.histogram(dr_dist,bins=200,range=(0,200))
                                hist_dr += h_dr[0]
                                
                            for rpt in range(0,len(r_home_subdiv)):
                                pos1=r_home_subdiv[rpt]
                                pos2=r_nn_subdiv[:].transpose()
                                r_dist=distance(pos1,pos2)
                                
                                h_rr = np.histogram(r_dist,bins=200,range=(0,200))
                                hist_rr += h_rr[0]
                                                 
end=time.time()
print("Done! Took %.2f seconds" % (end-start))

In [ ]:
norm = (((1e5)**2)-1e5)/2.0
dr_norm = ((1e5)**2)/1.0

zeta = (hist_dd/norm - (2*hist_dr/dr_norm) + hist_rr/norm)/(hist_rr/norm)

#zeta = (hist_dd[0] - 2*hist_dr[0] + hist_rr[0])/(hist_rr[0])


plt.figure()
plt.plot(zeta,'c-')
plt.plot(zeta,'mo',markersize=3)
sn.despine()



In [ ]:
len(dt[0])

In [ ]:
hist_dd